In [1]:
import re

import japanize_matplotlib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(font="IPAexGothic")

import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

import gc
from glob import glob
from functools import partial
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from xfeat import SelectCategorical, LabelEncoder, LambdaEncoder, Pipeline, ConcatCombination, SelectNumerical, \
    ArithmeticCombinations, TargetEncoder, aggregation, GBDTFeatureSelector, GBDTFeatureExplorer

In [2]:
train = pd.read_csv('output.csv')

/Users/otomoryosuke/opt/anaconda3/envs/Python37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
test = pd.read_csv('./data/test.csv')


In [4]:
test.isnull().sum()

ID                0
種類                0
地域            23151
市区町村コード           0
都道府県名             0
市区町村名             0
地区名               3
最寄駅：名称           32
最寄駅：距離（分）        32
間取り            1133
面積（㎡）             0
土地の形状         23151
間口            23151
延床面積（㎡）       23151
建築年             680
建物の構造          1374
用途             6704
今後の利用目的        1113
前面道路：方位       23151
前面道路：種類       23151
前面道路：幅員（ｍ）    23151
都市計画            293
建ぺい率（％）         405
容積率（％）          405
取引時点              0
改装             3104
取引の事情等        22907
dtype: int64

In [5]:
train.isnull().sum()

ID                   0
種類                   0
地域              677392
市区町村コード              0
都道府県名                0
市区町村名                0
地区名                660
最寄駅：名称            2672
最寄駅：距離（分）        23098
間取り              23661
面積（㎡）                0
土地の形状           677392
間口              677392
延床面積（㎡）         677392
建築年              19622
建物の構造            16486
用途               58246
今後の利用目的         364049
前面道路：方位         677392
前面道路：種類         677392
前面道路：幅員（ｍ）      677392
都市計画             19221
建ぺい率（％）          23345
容積率（％）           23345
取引時点                 0
改装               61966
取引の事情等          658854
取引価格（総額）_log         0
dtype: int64

In [6]:
ID = 'ID'
TARGET = '取引価格（総額）_log'
rm_cols = []

df = pd.concat([train, test])
rm_cols += ['市区町村コード']
for i, v in df.nunique().iteritems():
    if v <= 1:
        rm_cols.append(i)

rm_cols

['市区町村コード',
 '種類',
 '地域',
 '土地の形状',
 '間口',
 '延床面積（㎡）',
 '前面道路：方位',
 '前面道路：種類',
 '前面道路：幅員（ｍ）']

In [7]:
df.shape

(700543, 28)

In [8]:
def normalize_moyori(moyori):
    if moyori == moyori:
        if moyori == '30分?60分':
            moyori = 45
        elif moyori == '1H?1H30':
            moyori = 75
        elif moyori == '1H30?2H':
            moyori = 105
        elif moyori == '2H?':
            moyori = 120
        moyori = int(moyori)
    return moyori

In [9]:
def normalize_area(area):
    if area == area:
        area = int(re.sub('m\^2未満|㎡以上', '', str(area)))
    return area

In [10]:
def convert_wareki_to_seireki(wareki):
    if wareki == wareki:
        if wareki == '戦前':
            wareki = '昭和20年'
        value = wareki[2:-1]
        if value == '元':
            value = 1
        else:
            value = int(value)
        if '昭和' in wareki:
            seireki = 1925+value
        elif '平成' in wareki:
            seireki = 1988+value
        elif '令和' in wareki:
            seireki = 2018+value
    else:
        seireki = wareki
    return seireki

In [11]:
test[TARGET] = np.nan
train.drop(rm_cols, axis=1, inplace=True)
test.drop(columns=rm_cols, axis=1, inplace=True)


In [12]:
# df['取引時点_何年前'] = df['取引時点'].apply(lambda x: 2021-int(x[:4]))
df = pd.concat([train, test])
df.sort_values('取引時点', inplace=True)
df.reset_index(drop=True, inplace=True)
df.shape

(700543, 19)

In [13]:
df['取引時点'].unique()

array(['2005年第３四半期', '2005年第４四半期', '2006年第１四半期', '2006年第２四半期',
       '2006年第３四半期', '2006年第４四半期', '2007年第１四半期', '2007年第２四半期',
       '2007年第３四半期', '2007年第４四半期', '2008年第１四半期', '2008年第２四半期',
       '2008年第３四半期', '2008年第４四半期', '2009年第１四半期', '2009年第２四半期',
       '2009年第３四半期', '2009年第４四半期', '2010年第１四半期', '2010年第２四半期',
       '2010年第３四半期', '2010年第４四半期', '2011年第１四半期', '2011年第２四半期',
       '2011年第３四半期', '2011年第４四半期', '2012年第１四半期', '2012年第２四半期',
       '2012年第３四半期', '2012年第４四半期', '2013年第１四半期', '2013年第２四半期',
       '2013年第３四半期', '2013年第４四半期', '2014年第１四半期', '2014年第２四半期',
       '2014年第３四半期', '2014年第４四半期', '2015年第１四半期', '2015年第２四半期',
       '2015年第３四半期', '2015年第４四半期', '2016年第１四半期', '2016年第２四半期',
       '2016年第３四半期', '2016年第４四半期', '2017年第１四半期', '2017年第２四半期',
       '2017年第３四半期', '2017年第４四半期', '2018年第１四半期', '2018年第２四半期',
       '2018年第３四半期', '2018年第４四半期', '2019年第１四半期', '2019年第２四半期',
       '2019年第３四半期', '2019年第４四半期', '2020年第１四半期', '2020年第２四半期',
       '2020年第３四半期', '2020年第４四半期', '2021年第１四半期'], dtype

In [14]:
val_min_idx = min(df[df['取引時点'].str.contains('2020年第２四半期|2020年第３四半期', regex=True)].index)
test_min_idx = min(df[df['取引時点'].str.contains('2020年第４四半期|2021年第１四半期', regex=True)].index)
val_min_idx, test_min_idx

(652493, 677392)

In [15]:
set(df.iloc[val_min_idx:test_min_idx, :]['取引時点'].values)

{'2020年第２四半期', '2020年第３四半期'}

In [16]:
enc_dic = {}
for i, e in enumerate(sorted(list(set(df['取引時点'].values)))):
    enc_dic[e] = i
df['取引時点_enc'] = df['取引時点'].map(enc_dic)

In [17]:

df['建築年'] = df['建築年'].apply(lambda x: convert_wareki_to_seireki(x))
# df['取引_建築'] = df['取引時点'].apply(lambda x: int(x[:4])) - df['建築年']
df['面積（㎡）'] = df['面積（㎡）'].apply(lambda x: normalize_area(x))
df['最寄駅：距離（分）'] = df['最寄駅：距離（分）'].apply(lambda x: normalize_moyori(x))
df.drop(['取引時点'], axis=1, inplace=True)


In [18]:
# 今後の利用目的と建物の構造を削除
df.drop(['今後の利用目的', '建物の構造'], axis=1, inplace=True)


In [19]:
encoder = Pipeline([
    SelectCategorical(),
    LabelEncoder(output_suffix=""),
])

le_df = encoder.fit_transform(df)
le_df.head(2)

,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,用途,都市計画,改装,取引の事情等
0,0,0,0,0,0,0,-1,-1,-1
1,1,1,1,1,1,0,0,0,-1


In [20]:
num_feature = [x for x in df.columns if df[x].dtype != 'object']
df[num_feature].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700543 entries, 0 to 700542
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ID            700543 non-null  int64  
 1   最寄駅：距離（分）     677413 non-null  float64
 2   面積（㎡）         700543 non-null  int64  
 3   建築年           680241 non-null  float64
 4   建ぺい率（％）       676793 non-null  float64
 5   容積率（％）        676793 non-null  float64
 6   取引価格（総額）_log  677392 non-null  float64
 7   取引時点_enc      700543 non-null  int64  
dtypes: float64(5), int64(3)
memory usage: 42.8 MB


In [21]:
cat_feature = [x for x in df.columns if df[x].dtype == 'object']
df[cat_feature]

,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,用途,都市計画,改装,取引の事情等
0,神奈川県,横浜市南区,真金町,阪東橋,１Ｋ,住宅,NaN,NaN,NaN
1,東京都,中野区,中央,新中野,１ＬＤＫ,住宅,近隣商業地域,未改装,NaN
2,京都府,京都市伏見区,桃山町丹下,墨染,３ＬＤＫ,住宅,第１種住居地域,未改装,NaN
3,東京都,世田谷区,深沢,都立大学,３ＬＤＫ,住宅,第１種中高層住居専用地域,未改装,NaN
4,埼玉県,さいたま市緑区,原山,浦和,２ＬＤＫ,住宅,NaN,改装済,NaN
...,...,...,...,...,...,...,...,...,...
700538,愛知県,名古屋市千種区,桜が丘,星ケ丘(愛知),３ＬＤＫ,住宅,第１種住居地域,未改装,NaN
700539,東京都,新宿区,中落合,落合南長崎,１Ｋ,NaN,近隣商業地域,未改装,NaN
700540,東京都,新宿区,中落合,落合南長崎,１Ｋ,住宅,近隣商業地域,改装済,NaN
700541,愛知県,名古屋市千種区,末盛通,覚王山,１ＬＤＫ,住宅,商業地域,NaN,NaN


In [22]:
def get_agg_df(df, group_col):

    agg_df, agg_cols = aggregation(df,
                        group_key=group_col,
                        group_values=['最寄駅：距離（分）',
#                                       '面積（㎡）',
#                                       '建ぺい率（％）',
#                                       '容積率（％）'
                                     ],
                        agg_methods=['count', 'mean']
                        )

    return agg_df[agg_cols]

group_col = '市区町村名'
agg_dis_2 = get_agg_df(df, group_col)
group_col1 = '都道府県名'
agg_dis_pre = get_agg_df(df, group_col1)

In [23]:
df.head()

,ID,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,用途,都市計画,建ぺい率（％）,容積率（％）,改装,取引の事情等,取引価格（総額）_log,取引時点_enc
0,14198339,神奈川県,横浜市南区,真金町,阪東橋,NaN,１Ｋ,15,1992.0,住宅,NaN,NaN,NaN,NaN,NaN,6.698970,0
1,13318874,東京都,中野区,中央,新中野,NaN,１ＬＤＫ,40,1999.0,住宅,近隣商業地域,80.0,400.0,未改装,NaN,7.380211,0
2,26081806,京都府,京都市伏見区,桃山町丹下,墨染,4.0,３ＬＤＫ,60,1994.0,住宅,第１種住居地域,60.0,200.0,未改装,NaN,7.176091,0
3,13302280,東京都,世田谷区,深沢,都立大学,NaN,３ＬＤＫ,85,2004.0,住宅,第１種中高層住居専用地域,60.0,200.0,未改装,NaN,7.832509,0
4,11089836,埼玉県,さいたま市緑区,原山,浦和,NaN,２ＬＤＫ,50,1990.0,住宅,NaN,NaN,NaN,改装済,NaN,6.977724,0


In [24]:
feat_df = pd.concat([df[num_feature], le_df, agg_dis_2, agg_dis_pre], axis=1)

In [25]:
feat_df.head()

,ID,最寄駅：距離（分）,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,取引価格（総額）_log,取引時点_enc,都道府県名,市区町村名,...,最寄駅：名称,間取り,用途,都市計画,改装,取引の事情等,agg_count_最寄駅：距離（分）_grpby_市区町村名,agg_mean_最寄駅：距離（分）_grpby_市区町村名,agg_count_最寄駅：距離（分）_grpby_都道府県名,agg_mean_最寄駅：距離（分）_grpby_都道府県名
0,14198339,NaN,15,1992.0,NaN,NaN,6.698970,0,0,0,...,0,0,0,-1,-1,-1,3694,8.494857,95841,13.536514
1,13318874,NaN,40,1999.0,80.0,400.0,7.380211,0,1,1,...,1,1,0,0,0,-1,4524,6.402078,201943,7.800889
2,26081806,4.0,60,1994.0,60.0,200.0,7.176091,0,2,2,...,2,2,0,1,0,-1,1908,9.590147,15959,10.065982
3,13302280,NaN,85,2004.0,60.0,200.0,7.832509,0,1,3,...,3,2,0,2,0,-1,11256,8.792822,201943,7.800889
4,11089836,NaN,50,1990.0,NaN,NaN,6.977724,0,3,4,...,4,3,0,-1,1,-1,548,19.826642,41539,13.331857


In [26]:
feat_df['距離_bin'] = pd.cut(feat_df['最寄駅：距離（分）'], [0, 30, 45, 75, 120], labels=False)

In [27]:
nulti = feat_df['面積（㎡）'] * feat_df['容積率（％）']
feat_df['容積_面積'] = nulti

In [28]:
feat_df.head()

,ID,最寄駅：距離（分）,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,取引価格（総額）_log,取引時点_enc,都道府県名,市区町村名,...,用途,都市計画,改装,取引の事情等,agg_count_最寄駅：距離（分）_grpby_市区町村名,agg_mean_最寄駅：距離（分）_grpby_市区町村名,agg_count_最寄駅：距離（分）_grpby_都道府県名,agg_mean_最寄駅：距離（分）_grpby_都道府県名,距離_bin,容積_面積
0,14198339,NaN,15,1992.0,NaN,NaN,6.698970,0,0,0,...,0,-1,-1,-1,3694,8.494857,95841,13.536514,NaN,NaN
1,13318874,NaN,40,1999.0,80.0,400.0,7.380211,0,1,1,...,0,0,0,-1,4524,6.402078,201943,7.800889,NaN,16000.0
2,26081806,4.0,60,1994.0,60.0,200.0,7.176091,0,2,2,...,0,1,0,-1,1908,9.590147,15959,10.065982,0.0,12000.0
3,13302280,NaN,85,2004.0,60.0,200.0,7.832509,0,1,3,...,0,2,0,-1,11256,8.792822,201943,7.800889,NaN,17000.0
4,11089836,NaN,50,1990.0,NaN,NaN,6.977724,0,3,4,...,0,-1,1,-1,548,19.826642,41539,13.331857,NaN,NaN


In [29]:
feat_df['距離'] = feat_df['agg_count_最寄駅：距離（分）_grpby_都道府県名'] / (feat_df['agg_count_最寄駅：距離（分）_grpby_市区町村名']+1)

In [30]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(feat_df)

StandardScaler()

In [31]:
train_df = pd.DataFrame(sc.transform(feat_df.iloc[:val_min_idx, :]), columns=feat_df.columns)
val_df = pd.DataFrame(sc.transform(feat_df.iloc[val_min_idx:test_min_idx, :]), columns=feat_df.columns)
test_df = pd.DataFrame(sc.transform(feat_df.iloc[test_min_idx:, :]), columns=feat_df.columns)
print(train_df.shape, val_df.shape, test_df.shape)

(652493, 24) (24899, 24) (23151, 24)


In [32]:
feat_cols = [col for col in train_df.columns if col not in rm_cols+[ID, TARGET]]

In [33]:
train_df.dropna(inplace=True)

In [34]:
train_x = train_df[feat_cols]
train_y = train_df[TARGET]
val_x = val_df[feat_cols]
val_y = val_df[TARGET]
test_x = test_df[feat_cols]
test_y = test_df[TARGET]

In [35]:
train_x.isnull().sum()

最寄駅：距離（分）                          0
面積（㎡）                              0
建築年                                0
建ぺい率（％）                            0
容積率（％）                             0
取引時点_enc                           0
都道府県名                              0
市区町村名                              0
地区名                                0
最寄駅：名称                             0
間取り                                0
用途                                 0
都市計画                               0
改装                                 0
取引の事情等                             0
agg_count_最寄駅：距離（分）_grpby_市区町村名    0
agg_mean_最寄駅：距離（分）_grpby_市区町村名     0
agg_count_最寄駅：距離（分）_grpby_都道府県名    0
agg_mean_最寄駅：距離（分）_grpby_都道府県名     0
距離_bin                             0
容積_面積                              0
距離                                 0
dtype: int64

In [36]:
from sklearn.preprocessing import StandardScaler
# train_x= train_x.fillna(train_x.mean())

val_x = val_x.fillna(train_x.mean())

test_x = test_x.fillna(train_x.mean())

# 母集団は同じと仮定しているので、traindataの平均を使用

In [37]:
train_x.shape

(597393, 22)

In [42]:
def make_model(n_in, nh):
    inp = L.Input(name="inputs", shape=(n_in,))
    
    x = L.Dense(nh, activation="relu", name="d1")(inp)
    x = L.Dense(nh, activation="relu", name="d2")(x)
    x = L.Dense(nh, activation="relu", name="d3")(x)
    preds = L.Dense(1, activation="linear", name="preds")(x)
    
    model = M.Model(inp, preds, name="NN")
    model.compile(loss="mean_absolute_error", optimizer="adam")
    return model


In [43]:
net = make_model(train_x.shape[1], nh=8)
net.summary()

Model: "NN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 22)]              0         
_________________________________________________________________
d1 (Dense)                   (None, 8)                 184       
_________________________________________________________________
d2 (Dense)                   (None, 8)                 72        
_________________________________________________________________
d3 (Dense)                   (None, 8)                 72        
_________________________________________________________________
preds (Dense)                (None, 1)                 9         
Total params: 337
Trainable params: 337
Non-trainable params: 0
_________________________________________________________________


In [44]:
val_y.shape[0]

24899

In [46]:
oof = np.zeros(val_y.shape[0])
epochs = 100
nets = []

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3,  min_lr=0.0005)
es = EarlyStopping(monitor='val_loss', patience=3)
reg = make_model(train_x.shape[1], nh=8)
reg.fit(train_x, train_y, epochs=epochs, batch_size=64,
        validation_data=(val_x, val_y),
        verbose=1
)

oof = reg.predict(val_x, batch_size=100, verbose=1)
gc.collect()        
        

Epoch 1/100
9335/9335 [==============================] - 6s 626us/step - loss: 0.3494 - val_loss: 0.3458
Epoch 2/100
9335/9335 [==============================] - 6s 620us/step - loss: 0.3274 - val_loss: 0.3424
Epoch 3/100
9335/9335 [==============================] - 6s 619us/step - loss: 0.3245 - val_loss: 0.3382
Epoch 4/100
9335/9335 [==============================] - 6s 616us/step - loss: 0.3227 - val_loss: 0.3394
Epoch 5/100
9335/9335 [==============================] - 6s 614us/step - loss: 0.3210 - val_loss: 0.3400
Epoch 6/100
9335/9335 [==============================] - 6s 618us/step - loss: 0.3192 - val_loss: 0.3355
Epoch 7/100
9335/9335 [==============================] - 6s 618us/step - loss: 0.3177 - val_loss: 0.3331
Epoch 8/100
9335/9335 [==============================] - 6s 615us/step - loss: 0.3168 - val_loss: 0.3323
Epoch 9/100
9335/9335 [==============================] - 6s 620us/step - loss: 0.3163 - val_loss: 0.3422
Epoch 10/100
9335/9335 [==============================]

3902